In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip -q install findspark
!pip -q install pyspark

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 51.9 MB/s 


In [ ]:
import os
import json
import requests
import findspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

In [ ]:
import pyspark.sql.functions as f

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
with open('/content/areas.json','w') as f:
  json.dump(requests.get('https://api.hh.ru/areas').json(),f)

In [ ]:
df=spark.read.json('/content/areas.json')

In [ ]:
df.show()

+--------------------+----+--------------+---------+
|               areas|  id|          name|parent_id|
+--------------------+----+--------------+---------+
|[{[{[], 4228, Вил...| 113|        Россия|     null|
|[{[{[], 3331, Бар...|   5|       Украина|     null|
|[{[], 6251, Абай,...|  40|     Казахстан|     null|
|[{[], 2970, Агдаш...|   9|   Азербайджан|     null|
|[{[], 1007, Брест...|  16|      Беларусь|     null|
|[{[], 2820, Ахалц...|  28|        Грузия|     null|
|[{[], 2112, Абхаз...|1001|Другие регионы|     null|
|[{[], 2842, Балык...|  48|    Кыргызстан|     null|
|[{[], 2763, Аккур...|  97|    Узбекистан|     null|
+--------------------+----+--------------+---------+



In [ ]:
df.select(*['name']).show()

+--------------+
|          name|
+--------------+
|        Россия|
|       Украина|
|     Казахстан|
|   Азербайджан|
|      Беларусь|
|        Грузия|
|Другие регионы|
|    Кыргызстан|
|    Узбекистан|
+--------------+



In [ ]:
def nm(txt):
  return 1 if txt=='Россия' else 0
udf_nm=f.udf(nm,StringType())
df.withColumn('name1',udf_nm(df.name)).show()

+--------------------+----+--------------+---------+-----+
|               areas|  id|          name|parent_id|name1|
+--------------------+----+--------------+---------+-----+
|[{[{[], 4228, Вил...| 113|        Россия|     null|    1|
|[{[{[], 3331, Бар...|   5|       Украина|     null|    0|
|[{[], 6251, Абай,...|  40|     Казахстан|     null|    0|
|[{[], 2970, Агдаш...|   9|   Азербайджан|     null|    0|
|[{[], 1007, Брест...|  16|      Беларусь|     null|    0|
|[{[], 2820, Ахалц...|  28|        Грузия|     null|    0|
|[{[], 2112, Абхаз...|1001|Другие регионы|     null|    0|
|[{[], 2842, Балык...|  48|    Кыргызстан|     null|    0|
|[{[], 2763, Аккур...|  97|    Узбекистан|     null|    0|
+--------------------+----+--------------+---------+-----+



In [ ]:
with open('/content/py_area_1.json','w') as f:
  json.dump(requests.get('https://api.hh.ru/vacancies?text=Python&area=1&per_page=100').json()['items'],f)
df1=spark.read.json('/content/py_area_1.json')

with open('/content/py_area_2.json','w') as f:
  json.dump(requests.get('https://api.hh.ru/vacancies?text=Python&area=2&per_page=100').json()['items'],f)
df2=spark.read.json('/content/py_area_2.json')

In [ ]:
df1.schema

StructType([StructField('accept_temporary', BooleanType(), True), StructField('address', StructType([StructField('building', StringType(), True), StructField('city', StringType(), True), StructField('description', StringType(), True), StructField('id', StringType(), True), StructField('lat', DoubleType(), True), StructField('lng', DoubleType(), True), StructField('metro', StructType([StructField('lat', DoubleType(), True), StructField('line_id', StringType(), True), StructField('line_name', StringType(), True), StructField('lng', DoubleType(), True), StructField('station_id', StringType(), True), StructField('station_name', StringType(), True)]), True), StructField('metro_stations', ArrayType(StructType([StructField('lat', DoubleType(), True), StructField('line_id', StringType(), True), StructField('line_name', StringType(), True), StructField('lng', DoubleType(), True), StructField('station_id', StringType(), True), StructField('station_name', StringType(), True)]), True), True), Stru

In [ ]:
df1.columns

['accept_temporary',
 'address',
 'adv_response_url',
 'alternate_url',
 'apply_alternate_url',
 'archived',
 'area',
 'contacts',
 'created_at',
 'department',
 'employer',
 'has_test',
 'id',
 'insider_interview',
 'name',
 'premium',
 'published_at',
 'relations',
 'response_letter_required',
 'response_url',
 'salary',
 'schedule',
 'snippet',
 'sort_point_distance',
 'type',
 'url',
 'working_days',
 'working_time_intervals',
 'working_time_modes']

In [ ]:
df1.join(df2,df1.id==df2.id,'outer').count()

200

In [ ]:
df3=df1.join(df2,df1.id==df2.id,'outer',)

In [ ]:
df3.columns

['accept_temporary',
 'address',
 'adv_response_url',
 'alternate_url',
 'apply_alternate_url',
 'archived',
 'area',
 'contacts',
 'created_at',
 'department',
 'employer',
 'has_test',
 'id',
 'insider_interview',
 'name',
 'premium',
 'published_at',
 'relations',
 'response_letter_required',
 'response_url',
 'salary',
 'schedule',
 'snippet',
 'sort_point_distance',
 'type',
 'url',
 'working_days',
 'working_time_intervals',
 'working_time_modes',
 'accept_temporary',
 'address',
 'adv_response_url',
 'alternate_url',
 'apply_alternate_url',
 'archived',
 'area',
 'contacts',
 'created_at',
 'department',
 'employer',
 'has_test',
 'id',
 'insider_interview',
 'name',
 'premium',
 'published_at',
 'relations',
 'response_letter_required',
 'response_url',
 'salary',
 'schedule',
 'snippet',
 'sort_point_distance',
 'type',
 'url',
 'working_days',
 'working_time_intervals',
 'working_time_modes']

In [ ]:
df1.select(*['name','id']).union(df2.select(*['name','id'])).count()

200

In [ ]:
df1.union(df2)

AnalysisException: ignored

In [ ]:
df1.show(3)

+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------+--------------------+----------+--------------------+--------+--------+-----------------+--------------------+-------+--------------------+---------+------------------------+------------+--------------------+--------------------+--------------------+-------------------+----------------+--------------------+------------+----------------------+------------------+
|accept_temporary|             address|    adv_response_url|       alternate_url| apply_alternate_url|archived|                area|contacts|          created_at|department|            employer|has_test|      id|insider_interview|                name|premium|        published_at|relations|response_letter_required|response_url|              salary|            schedule|             snippet|sort_point_distance|            type|                 url|working_days|working_time_intervals|working_ti

In [ ]:
df1.select(*['insider_interview']).show(100)

+-----------------+
|insider_interview|
+-----------------+
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|


In [ ]:
df2.select(*['insider_interview']).show(100)

+--------------------+
|   insider_interview|
+--------------------+
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|          

In [ ]:
df2.schema

StructType([StructField('accept_temporary', BooleanType(), True), StructField('address', StructType([StructField('building', StringType(), True), StructField('city', StringType(), True), StructField('description', StringType(), True), StructField('id', StringType(), True), StructField('lat', DoubleType(), True), StructField('lng', DoubleType(), True), StructField('metro', StructType([StructField('lat', DoubleType(), True), StructField('line_id', StringType(), True), StructField('line_name', StringType(), True), StructField('lng', DoubleType(), True), StructField('station_id', StringType(), True), StructField('station_name', StringType(), True)]), True), StructField('metro_stations', ArrayType(StructType([StructField('lat', DoubleType(), True), StructField('line_id', StringType(), True), StructField('line_name', StringType(), True), StructField('lng', DoubleType(), True), StructField('station_id', StringType(), True), StructField('station_name', StringType(), True)]), True), True), Stru